In [76]:
cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/

/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [84]:
train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/Train_FeatureEngineering.csv")
print("Shape of Training Data",train.shape)
test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/Test_FeatureEngineering.csv")
print("Shape of Testing Data", test.shape)


Shape of Training Data (17170, 25)
Shape of Testing Data (8576, 25)


In [0]:
train.info()

# Data_Preprocessing

In [0]:
def encodeDays(day_of_week):
  day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
  return day_dict[day_of_week]
def encodeLabel(label):
  label_dict={'correct':1,'incorrect':0}
  return label_dict[label]
def clean_data(data,isTrain):
  data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
  if(isTrain):
    data['label']=data['label'].apply(lambda x:encodeLabel(x))
  return data


In [86]:
train=clean_data(train,True)
test=clean_data(test,False)
print("Shape of Training Data after cleaning ",train.shape)
print("Shape of Testing Data after cleaning", test.shape)

Shape of Training Data after cleaning  (17170, 25)
Shape of Testing Data after cleaning (8576, 25)


In [0]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3',]
drop_columns_test=['tripid','pickup_time','drop_time','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

In [88]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (17170, 17)
Shape of Testing Data after dropping columns (8576, 16)


In [89]:
train_1=pd.get_dummies(train_1)
test_1=pd.get_dummies(test_1)

print("Shape of Training Data after One Hot Encoding",train_1.shape)
print("Shape of Testing Data after One Hot Encoding",test_1.shape)

Shape of Training Data after One Hot Encoding (17170, 17)
Shape of Testing Data after One Hot Encoding (8576, 16)


In [0]:
train_1 = train_1.dropna()

In [0]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [92]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13569
Number of records in validation data  3393


# XGBoost CLassifier

In [93]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [97]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.08%


In [0]:
y_pred = model.predict(test_1)
predictions = [round(value) for value in y_pred]

# Linear Regression Classifier

In [67]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=lm.predict(X_test)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)


RMSE for Linear Regression is  0.2939505199843409


# Get Predictions

In [0]:
y_pred=lm.predict(test_1)

In [0]:
 y_pred = pd.DataFrame({'label':predictions})

In [0]:
 y_pred["label"] = y_pred["label"].astype(int)

In [0]:
y_pred['label'] = y_pred['label'].apply(lambda x: 0 if x < 0.5 else 1)

In [0]:
y_pred.to_csv("predications_xgboost.csv")